In [2]:
import pandas as pd


Create csv file that for each year has a row per team, and includes their average per game stats

In [3]:
# Load the dataset
file_path = "../data/MRegularSeasonDetailedResults.csv"  # Update with the actual file path
df = pd.read_csv(file_path)

# Load the MMasseyOrdinals file
massey_file_path = "../data/MMasseyOrdinals.csv"  # Update with the actual file path
massey_df = pd.read_csv(massey_file_path)


team_spellings = pd.read_csv('../data/MTeamSpellings.csv')  # Replace with the actual path to the team spellings file



# Filter for POM rankings on day 128 for the current season
current_season = df["Season"].max()  # Get the current season (latest season in the data)
pom_rankings = massey_df[
    (massey_df["Season"] == current_season) & 
    (massey_df["RankingDayNum"] == 128) & 
    (massey_df["SystemName"] == "POM")
]

# Create a new DataFrame to store team stats
team_stats = {}

# Process each game
for _, row in df.iterrows():
    season = row["Season"]
    
    # Winning team stats
    w_team = row["WTeamID"]
    if (season, w_team) not in team_stats:
        team_stats[(season, w_team)] = {
            "Season": season, "TeamID": w_team, "Games": 0, "Wins": 0,
            "Points": 0, "OppPoints": 0, "FGM": 0, "FGA": 0, "FGM3": 0, "FGA3": 0,
            "FTM": 0, "FTA": 0, "OREB": 0, "DREB": 0, "Turnovers": 0, "Steals": 0, "Blocks": 0
        }
    
    team_stats[(season, w_team)]["Games"] += 1
    team_stats[(season, w_team)]["Wins"] += 1
    team_stats[(season, w_team)]["Points"] += row["WScore"]
    team_stats[(season, w_team)]["OppPoints"] += row["LScore"]
    team_stats[(season, w_team)]["FGM"] += row["WFGM"]
    team_stats[(season, w_team)]["FGA"] += row["WFGA"]
    team_stats[(season, w_team)]["FGM3"] += row["WFGM3"]
    team_stats[(season, w_team)]["FGA3"] += row["WFGA3"]
    team_stats[(season, w_team)]["FTM"] += row["WFTM"]
    team_stats[(season, w_team)]["FTA"] += row["WFTA"]
    team_stats[(season, w_team)]["OREB"] += row["WOR"]
    team_stats[(season, w_team)]["DREB"] += row["WDR"]
    team_stats[(season, w_team)]["Turnovers"] += row["WTO"]
    team_stats[(season, w_team)]["Steals"] += row["WStl"]
    team_stats[(season, w_team)]["Blocks"] += row["WBlk"]

    # Losing team stats
    l_team = row["LTeamID"]
    if (season, l_team) not in team_stats:
        team_stats[(season, l_team)] = {
            "Season": season, "TeamID": l_team, "Games": 0, "Wins": 0,
            "Points": 0, "OppPoints": 0, "FGM": 0, "FGA": 0, "FGM3": 0, "FGA3": 0,
            "FTM": 0, "FTA": 0, "OREB": 0, "DREB": 0, "Turnovers": 0, "Steals": 0, "Blocks": 0
        }
    
    team_stats[(season, l_team)]["Games"] += 1
    team_stats[(season, l_team)]["Points"] += row["LScore"]
    team_stats[(season, l_team)]["OppPoints"] += row["WScore"]
    team_stats[(season, l_team)]["FGM"] += row["LFGM"]
    team_stats[(season, l_team)]["FGA"] += row["LFGA"]
    team_stats[(season, l_team)]["FGM3"] += row["LFGM3"]
    team_stats[(season, l_team)]["FGA3"] += row["LFGA3"]
    team_stats[(season, l_team)]["FTM"] += row["LFTM"]
    team_stats[(season, l_team)]["FTA"] += row["LFTA"]
    team_stats[(season, l_team)]["OREB"] += row["LOR"]
    team_stats[(season, l_team)]["DREB"] += row["LDR"]
    team_stats[(season, l_team)]["Turnovers"] += row["LTO"]
    team_stats[(season, l_team)]["Steals"] += row["LStl"]
    team_stats[(season, l_team)]["Blocks"] += row["LBlk"]

# Convert dictionary to DataFrame
teams_df = pd.DataFrame.from_dict(team_stats, orient="index")

# Calculate per-game statistics
teams_df["Win%"] = (teams_df["Wins"] / teams_df["Games"]).round(3)
teams_df["PPG"] = (teams_df["Points"] / teams_df["Games"]).round(3)
teams_df["OPPG"] = (teams_df["OppPoints"] / teams_df["Games"]).round(3)
teams_df["FG%"] = (teams_df["FGM"] / teams_df["FGA"]).round(3)
teams_df["FGA/G"] = (teams_df["FGA"] / teams_df["Games"]).round(3)  # FGA per game
teams_df["3P%"] = (teams_df["FGM3"] / teams_df["FGA3"]).round(3)
teams_df["FGA3/G"] = (teams_df["FGA3"] / teams_df["Games"]).round(3)  # FGA3 per game
teams_df["FT%"] = (teams_df["FTM"] / teams_df["FTA"]).round(3)
teams_df["RPG"] = ((teams_df["OREB"] + teams_df["DREB"]) / teams_df["Games"]).round(3)
teams_df["TPG"] = (teams_df["Turnovers"] / teams_df["Games"]).round(3)
teams_df["SPG"] = (teams_df["Steals"] / teams_df["Games"]).round(3)
teams_df["BPG"] = (teams_df["Blocks"] / teams_df["Games"]).round(3)

# Calculate Pace
teams_df["Pace"] = (
    (teams_df["FGA"] + 0.475 * teams_df["FTA"] - teams_df["OREB"] + teams_df["Turnovers"]) / teams_df["Games"]
).round(3)

# Merge POM rankings into the teams_df
teams_df = teams_df.merge(
    pom_rankings[["TeamID", "OrdinalRank"]],
    how="left",
    on="TeamID"
)

# Rename the POM ranking column for clarity
teams_df.rename(columns={"OrdinalRank": "POM_Rank"}, inplace=True)


# Create a dictionary to map TeamID to TeamNameSpelling
team_id_to_name = dict(zip(team_spellings['TeamID'], team_spellings['TeamNameSpelling']))

# Map TeamID to TeamNameSpelling
teams_df['Team'] = teams_df['TeamID'].map(team_id_to_name)

# If you want the team names in lowercase (as your original code suggested)
teams_df['Team'] = teams_df['Team'].str.lower()

# Select only the required columns
teams_df = teams_df[
    ["Season", "Team", "TeamID", "Win%", "PPG", "OPPG", "FG%", "FGA/G", "3P%", "FGA3/G", "FT%", "RPG", "TPG", "SPG", "BPG", "Pace", "POM_Rank"]
]

print(teams_df.head())

# Save to CSV
output_path = "class_Data/team_season_stats.csv"
teams_df.to_csv(output_path, index=False)

print("Processing complete. File saved as:", output_path)

   Season       Team  TeamID   Win%     PPG    OPPG    FG%   FGA/G    3P%  \
0    2003    alabama    1104  0.607  69.286  65.000  0.420  57.179  0.320   
1    2003   oklahoma    1328  0.800  71.167  60.167  0.447  56.533  0.394   
2    2003    memphis    1272  0.793  74.517  65.828  0.438  60.000  0.349   
3    2003   syracuse    1393  0.828  80.103  69.897  0.470  62.207  0.330   
4    2003  marquette    1266  0.821  78.393  67.679  0.484  56.250  0.379   

   FGA3/G    FT%     RPG     TPG    SPG    BPG    Pace  POM_Rank  
0  19.857  0.710  37.500  13.286  6.607  3.786  66.834       6.0  
1  18.967  0.708  37.100  11.800  6.933  3.767  65.035      38.0  
2  20.069  0.654  40.034  13.793  7.379  5.069  70.600      48.0  
3  15.862  0.693  41.207  13.621  8.310  7.276  72.737     112.0  
4  15.250  0.770  37.179  13.571  6.000  3.643  67.928      26.0  
Processing complete. File saved as: class_Data/team_season_stats.csv


For every game in the data set, pull each team's respective stats from that season, and calculate the difference of each stat and add that to the row including who won so a model can use those columns as features, and W/S as the target

In [4]:
games_file_path = '../data/MRegularSeasonCompactResults.csv'
stats_file_path = 'class_data/team_season_stats.csv'

games_df = pd.read_csv(games_file_path)
stats_df = pd.read_csv(stats_file_path)


# Step 2: Filter the game results to include only seasons from 2003 onwards
games_df = games_df[games_df['Season'] >= 2003]

# Step 3: Prepare separate rows for the winning and losing teams
# Create a row for the winning team
win_df = games_df[['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore']].copy()
win_df['target'] = 1  # Winning team gets target = 1
win_df['TeamID'] = win_df['WTeamID']
win_df['OpponentID'] = win_df['LTeamID']
win_df['Score'] = win_df['WScore']
win_df['OpponentScore'] = win_df['LScore']

# Create a row for the losing team
lose_df = games_df[['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore']].copy()
lose_df['target'] = 0  # Losing team gets target = 0
lose_df['TeamID'] = lose_df['LTeamID']
lose_df['OpponentID'] = lose_df['WTeamID']
lose_df['Score'] = lose_df['LScore']
lose_df['OpponentScore'] = lose_df['WScore']

# Combine the winning and losing data into one dataframe
game_df = pd.concat([win_df, lose_df], axis=0, ignore_index=True)

# Step 4: Merge with team stats
game_df = pd.merge(game_df, stats_df, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'], suffixes=('_Team', '_Opponent'))

# Now merge for the opponent stats
game_df = pd.merge(game_df, stats_df, left_on=['Season', 'OpponentID'], right_on=['Season', 'TeamID'], suffixes=('_Team', '_Opponent'))

game_df['diff_PPG'] = game_df['PPG_Team'] - game_df['PPG_Opponent']
game_df['diff_OPPG'] = game_df['OPPG_Team'] - game_df['OPPG_Opponent']
game_df['diff_FG%'] = game_df['FG%_Team'] - game_df['FG%_Opponent']
game_df['diff_FGA/G'] = game_df['FGA/G_Team'] - game_df['FGA/G_Opponent']
game_df['diff_3P%'] = game_df['3P%_Team'] - game_df['3P%_Opponent']
game_df['diff_FGA3/G'] = game_df['FGA3/G_Team'] - game_df['FGA3/G_Opponent']
game_df['diff_FT%'] = game_df['FT%_Team'] - game_df['FT%_Opponent']
game_df['diff_RPG'] = game_df['RPG_Team'] - game_df['RPG_Opponent']
game_df['diff_TPG'] = game_df['TPG_Team'] - game_df['TPG_Opponent']
game_df['diff_SPG'] = game_df['SPG_Team'] - game_df['SPG_Opponent']
game_df['diff_BPG'] = game_df['BPG_Team'] - game_df['BPG_Opponent']
game_df['diff_Pace'] = game_df['Pace_Team'] - game_df['Pace_Opponent']
game_df['diff_POM_Rank'] = game_df['POM_Rank_Team'] - game_df['POM_Rank_Opponent']


# Round the differences to 3 decimal points
diff_columns = [
    'diff_PPG', 'diff_OPPG',
    'diff_FG%', 'diff_FGA/G', 'diff_3P%', 'diff_FGA3/G', 'diff_FT%', 'diff_RPG', 'diff_TPG',
    'diff_SPG', 'diff_BPG', 'diff_Pace', 'diff_POM_Rank'
]

game_df[diff_columns] = game_df[diff_columns].round(3)

# Step 6: Save the merged and processed data to a CSV
game_df.to_csv('class_data/reg_season_final.csv', index=False)



Do the same, but for the tournament games to act as a train or test set

In [5]:
games_file_path = '../data/MNCAATourneyCompactResults.csv'
stats_file_path = 'class_data/team_season_stats.csv'

games_df = pd.read_csv(games_file_path)
stats_df = pd.read_csv(stats_file_path)


# Step 2: Filter the game results to include only seasons from 2003 onwards
games_df = games_df[games_df['Season'] >= 2003]

# Step 3: Prepare separate rows for the winning and losing teams
# Create a row for the winning team
win_df = games_df[['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore']].copy()
win_df['target'] = 1  # Winning team gets target = 1
win_df['TeamID'] = win_df['WTeamID']
win_df['OpponentID'] = win_df['LTeamID']
win_df['Score'] = win_df['WScore']
win_df['OpponentScore'] = win_df['LScore']

# Create a row for the losing team
lose_df = games_df[['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore']].copy()
lose_df['target'] = 0  # Losing team gets target = 0
lose_df['TeamID'] = lose_df['LTeamID']
lose_df['OpponentID'] = lose_df['WTeamID']
lose_df['Score'] = lose_df['LScore']
lose_df['OpponentScore'] = lose_df['WScore']

# Combine the winning and losing data into one dataframe
game_df = pd.concat([win_df, lose_df], axis=0, ignore_index=True)

# Step 4: Merge with team stats
game_df = pd.merge(game_df, stats_df, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'], suffixes=('_Team', '_Opponent'))

# Now merge for the opponent stats
game_df = pd.merge(game_df, stats_df, left_on=['Season', 'OpponentID'], right_on=['Season', 'TeamID'], suffixes=('_Team', '_Opponent'))


game_df['diff_PPG'] = game_df['PPG_Team'] - game_df['PPG_Opponent']
game_df['diff_OPPG'] = game_df['OPPG_Team'] - game_df['OPPG_Opponent']
game_df['diff_FG%'] = game_df['FG%_Team'] - game_df['FG%_Opponent']
game_df['diff_FGA/G'] = game_df['FGA/G_Team'] - game_df['FGA/G_Opponent']
game_df['diff_3P%'] = game_df['3P%_Team'] - game_df['3P%_Opponent']
game_df['diff_FGA3/G'] = game_df['FGA3/G_Team'] - game_df['FGA3/G_Opponent']
game_df['diff_FT%'] = game_df['FT%_Team'] - game_df['FT%_Opponent']
game_df['diff_RPG'] = game_df['RPG_Team'] - game_df['RPG_Opponent']
game_df['diff_TPG'] = game_df['TPG_Team'] - game_df['TPG_Opponent']
game_df['diff_SPG'] = game_df['SPG_Team'] - game_df['SPG_Opponent']
game_df['diff_BPG'] = game_df['BPG_Team'] - game_df['BPG_Opponent']
game_df['diff_Pace'] = game_df['Pace_Team'] - game_df['Pace_Opponent']
game_df['diff_POM_Rank'] = game_df['POM_Rank_Team'] - game_df['POM_Rank_Opponent']


# Round the differences to 3 decimal points
diff_columns = [
    'diff_PPG', 'diff_OPPG',
    'diff_FG%', 'diff_FGA/G', 'diff_3P%', 'diff_FGA3/G', 'diff_FT%', 'diff_RPG', 'diff_TPG',
    'diff_SPG', 'diff_BPG', 'diff_Pace', 'diff_POM_Rank'
]

game_df[diff_columns] = game_df[diff_columns].round(3)

# Step 6: Save the merged and processed data to a CSV
game_df.to_csv('class_data/tourney_final.csv', index=False)